# Notebook - FastF1

Notebook donde se prueba el paquete de Python FastF1, el cual permite acceder y analizar datos de la Formula 1. FastF1 se utiliza como complemento a la API de Ergast para aumentar el conjunto de datos con el que trabajar en el proyecto. La obtención de los datos para este caso se hará en tiempo real, haciendo uso de las funciones que ofrece el paquete, que permite hacer consultas por temporada y gran premio.

Importamos bibliotecas necesarias

In [ ]:
import requests
import csv
import pandas as pd
import json
import ast
import fastf1
import fastf1.plotting

### Stints
Realizamos una primera consulta para extrar los datos para el Gran Premio de Hungría para el año 2022 y sacamos por pantalla varios de los datos que devuelve la consulta

In [ ]:
# Creamos el objeto Session para la carrera de Hungría de 2022, como se especifica en los parámetros. Llamamos posteriormente a load() para cargar los datos del objeto Session
session = fastf1.get_session(2022, 'Hungary', 'R')
session.load()

# Obtener la información relativa a las vueltas
laps = session.laps

# Obtener información sobre los pilotos
drivers = session.drivers

# Obtener los resultados
results = session.results

# Crear una lista con las abreviaciones de los nombres de los pilotos
drivers = [session.get_driver(driver)["Abbreviation"] for driver in drivers]

# 
# positions = results[["DriverNumber", "Abbreviation", "GridPosition", "Position"]]
# positions_list = positions['Abbreviation'].to_list()

# Contabilizar el número de stints por pilotos y el compuesto utilizado agrupando por columnas y contando los registros generados
stints = laps[["Driver", "DriverNumber", "Stint", "Compound", "LapNumber"]]
stints = stints.groupby(["Driver", "DriverNumber", "Stint", "Compound"])
stints = stints.count().reset_index()

# Renombrar columnas
stints = stints.rename(columns={"LapNumber": "StintLength"})


Resultado final del dataframe stints

In [ ]:
stints.head()

Visualizamos datos del dataframe laps

In [ ]:
laps.head()

In [ ]:
laps.tail()

Borramos columnas innecesarias del dataframe

In [ ]:
laps_updated = laps.drop(columns=['Time', 'PitOutTime', 'PitInTime', 'FastF1Generated', 'FreshTyre', 'IsAccurate', 'Deleted', 'DeletedReason'], axis=1)
laps_updated.head()

In [ ]:
sorted_positions = laps_updated.sort_values(by='Position', ascending=True)
sorted_positions.head()

### Representaciones gráficas de los datos extraidos

#### Gráfico de líneas: Cambio de posiciones
Gráfico con el que se pretende representar los cambios de posiciones de los pilotos en el transcurso de la carrera, donde el eje X representa el número de vuelta e Y la posición. Se utiliza un gráfico de líneas con la librería Plotly.

In [ ]:
import plotly.express as px
# Gráfico de líneas
fig = px.line(laps_updated, 
            x='LapNumber', 
            y='Position', 
            color='Driver', 
            markers=True)
            
fig.update_layout(
    yaxis=dict(autorange="reversed")  # Invertir el orden del eje Y
)
# Mostrar el gráfico
fig.show()


#### Gráfico de barras: Visualizar balance de posición de salida vs. posición final
Gráfico de barras con el que se pretende analizar las posiciones ganadas y/o perdidas por los pilotos durante una carrera. Para ello, se añade una nueva columna al dataset a usar con el resultado de calcular la diferencia entre la posición de salida y la posición final.

In [ ]:
positions.head()

Calculamos el número de posiciones ganadas y/o perdidas considerando la posición de inicio de carrera de cada piloto y la posición final.

In [ ]:
positions_df = positions.copy()
positions_df['PositionChange'] = positions_df['GridPosition'] - positions_df['Position']
positions_df.head()


Utilizamos la librería plotly para obtener una visualización de muestra de la diferencia de posiciones, donde el eje X representa la diferenica de las posiciones y el eje Y cada piloto.

In [ ]:
# Gráfico de barras
fig = px.bar(
    positions_df,
    x='PositionChange',
    y='Abbreviation',
    color='PositionChange',
    orientation='h',
    title="Ganancia o Pérdida de Posiciones por Piloto"
)
fig.show()

#### Gráfico de violines: Visualizar los tiempos de vueltas por piloto
Gráfico de violines con el que se pretende analizar los tiempos de vueltas por piloto durante la carrera y analizar la consistencia de los mismos. Los puntos que representan cada data también son dibujados en el gráfico para visualizar la distribución de los distintos tiempos.

In [ ]:
# Convertir tiempos de vueltas a segundos
laps_updated['LapTimeSeconds'] = laps_updated['LapTime'].dt.total_seconds()

# Obtener los nombres abreviados de los pilotos ordenados por posición final en formato lista, con el objetivo de dibujar los violines en el mismo orden
positions_list = positions['Abbreviation'].to_list()

# Gráfico de violines
fig = px.violin(
    laps_updated,
    x='Driver',
    y='LapTimeSeconds',
    category_orders={'Driver': positions_list}, # Ordenar pilotos en el eje X
    box=True,
    points="all"
)
fig.show()

In [ ]:
laps_updated.head(5)

#### Gráfico de barras: Tiempo de vuelta rápida
En el siguiente gráfico de barras verticales se puede analizar el tiempo de vuelta rápida por cada piloto en segundos. 

In [ ]:
# Coger la fila con el tiempo de vuelta en segundos inferior
fastest_laps = laps_updated.groupby('Driver')['LapTimeSeconds'].min().reset_index()

# Gráfico de barras
fig = px.bar(
    fastest_laps,
    x='Driver',
    y='LapTimeSeconds',
    color='Driver',
    category_orders={'Driver': positions_list}, # Ordenar pilotos en el eje X
    title="Vueltas Más Rápidas por Piloto",
)
# Acotar los valores del eje X
fig.update_yaxes(range=[60,110])
fig.show()

#### Stints
En esta sección se pretende hacer una representación del uso que ha hecho cada piloto de los distintos compuestos de neumáticos, así como el número de vueltas realizados en cada stint con un mismo compuesto. Permite hacernos una idea de la estrategia de neumáticos utilizada por cada piloto.

En primer lugar, se concatena el dataframe stints con el dataframe positions_df. Queremos mostrar la posición de parrilla y la posición final en el hover text de cara barra.

In [ ]:
stints.head()

In [ ]:
positions_df.head()

In [ ]:
stints_with_position = pd.merge(stints, positions_df, on=['DriverNumber'], how='left')
stints_with_position = stints_with_position.astype({'Position':'int64', 'GridPosition':'int64'})
stints_with_position = stints_with_position.sort_values(by=['Position', 'Stint'])
stints_with_position.head()

Una vez tenemos el conjunto de datos preparado, se procede a crear el gráfico. En el código fuente se han añadido notas para entender cómo se ha implementado.

In [ ]:
import plotly.graph_objects as go # Módulo graph objects de Plotly
import numpy as np

# Crear una figura en Plotly
fig = go.Figure()

compounds_set = set() # Inicializar set
# Iterar por cada piloto
for driver in drivers:
    # Obtener los stints del piloto
    driver_stints = stints_with_position.loc[stints["Driver"] == driver].copy()

    # Reemplazar los valores de las columnas Stint y Compound que tienen un valor string 'nan' por el NaN de Numpy
    driver_stints[['Stint', 'Compound']] = driver_stints[['Stint', 'Compound']].replace('nan', np.nan)
    # Rellenar los posibles valores faltantes por el valor del registro siguiente, que coincidirá en Stint y Compound
    driver_stints[['Stint', 'Compound']] = driver_stints[['Stint', 'Compound']].bfill()
    
    previous_stint_end = 0 # Inicializar variable que permitirá establecer el inicio de la barra en el gráfico para representar los stints
    for _, row in driver_stints.iterrows():
        # Obtener el color correspondiente al compuesto con la función que ofrece la propia librería
        compound_color = fastf1.plotting.get_compound_color(row["Compound"], session=session)
        compounds_set.add((row["Compound"],compound_color)) # Añadir una tupla que será el compuesto y su color

        # Añadir una barra horizontal para cada stint del piloto
        fig.add_trace(go.Bar(
            y=[driver],  # Eje Y muestra al piloto
            x=[row["StintLength"]],  # Eje X es la duración del stint
            base=previous_stint_end,  # El inicio de la barra
            orientation='h',  # Barras horizontales
            marker=dict(color=compound_color),  # Colores y bordes
            name=row["Compound"],  # Nombre del compuesto para el hover y contenido del hover
            hovertemplate=(
                f"Piloto: {driver}<br>" +
                f"Compuesto: {row['Compound']}<br>" +
                f"Duración del stint: {row['StintLength']} vueltas<br>" +
                f"Inicia en la vuelta: {previous_stint_end}<br>"+
                f"Posicion inicial: {row['GridPosition']}<br>"+
                f"Posicion final: {row['Position']}<br>"
            ),
            showlegend=False 
        ))

        previous_stint_end += row["StintLength"] # Actualizar variable con la duración del stint para conocer donde debe empezar la siguiente barra horizontal

# Crear la leyenda
for compound, color in compounds_set:
    fig.add_trace(go.Bar(
        y=[None],  # Para no mostrar una barra visible, solo usar la leyenda
        x=[0],  # Valor de 0 para que no se dibuje una barra visible
        name=compound,  # Nombre de la entrada de la leyenda
        marker=dict(color=color),  # El color correspondiente al compuesto
        showlegend=True  # Mostrar una entrada en la leyenda para cada compuesto
))

# Configuración del diseño
fig.update_layout(
    title="2022 Hungarian Grand Prix Strategies",
    height=800,
    xaxis_title="Lap Number",
    yaxis_title="Driver",
    barmode='stack',  # Las barras se apilan horizontalmente
    xaxis=dict(showgrid=False, zeroline=False),  # Ocultar líneas de rejilla
    yaxis=dict(autorange="reversed")  # Invertir el orden de los pilotos
)

# Mostrar el gráfico
fig.show()


#### Gráfico de barras: Número de vueltas promedio por compuesto
Sección para visualizar mediante un gráfico de barras el número promedio de vueltas realizados por los pilotos de media. El objetivo es tener en el eje X el tipo de compuesto y en el eje Y el número de vueltas promedio por compuesto.

Contabiliazamos el número de vueltas dado por pilotos por compuesto. Para ello, se genera un nuevo dataframe a partir de la agrupación de los registros, como se puede observar en la siguiente celda.

In [ ]:
laps_per_stint_compound = laps_updated.groupby(['Driver', 'Stint', 'Compound'])['LapNumber'].count().reset_index()
laps_per_stint_compound.rename({'LapNumber': 'LapCount'}, inplace=True)
laps_per_stint_compound.head()

Calculamos la media de vueltas por compuesto

In [ ]:
laps_compound_mean_df = laps_per_stint_compound.groupby('Compound')['LapNumber'].mean().reset_index()

Utilizamos plotly para generar el gráfico de barras. Cada barra representa un compuesto y se utiliza el color que representa a cada compuesto, obtenido con anterioridad de la librería.

In [ ]:

fig = px.bar(
    laps_compound_mean_df,
    x='Compound',
    y='LapNumber',
    color='Compound',
    title=f'Número de vueltas por compuesto y stint',
    labels={'LapCount': 'Número de vueltas', 'Compound': 'Compuesto', 'Stint': 'Tanda'},
    color_discrete_map=compound_colors
)

fig.show()